In [38]:
import time
import tensorflow as tf
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data

# Read data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

#Set a fixed random seed
tf.set_random_seed(0);

#Initialise in order to save the neural network for space computation
saver = tf.train.Saver()

#Are we using dropout??? What about the original AlexNet?

# Parameters
learning_rate = 0.001
base_training_iters = 200
batch_size = 64
display_step = 20
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 1 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

#Iterator number
def itr_num(x):
    count=-1;
    while(x>=1):
        a = x%10;
        x = x/10;
        count += 1;
    return round(a)*10**count;

# Create AlexNet model
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

def alex_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    pool1 = max_pool('pool1', conv1, k=2)
    # Apply Normalization
    norm1 = norm('norm1', pool1, lsize=4)
    # Apply Dropout
    norm1 = tf.nn.dropout(norm1, _dropout)

    # Convolution Layer
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    pool2 = max_pool('pool2', conv2, k=2)
    # Apply Normalization
    norm2 = norm('norm2', pool2, lsize=4)
    # Apply Dropout
    norm2 = tf.nn.dropout(norm2, _dropout)

    # Convolution Layer
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # Max Pooling (down-sampling)
    pool3 = max_pool('pool3', conv3, k=2)
    # Apply Normalization
    norm3 = norm('norm3', pool3, lsize=4)
    # Apply Dropout
    norm3 = tf.nn.dropout(norm3, _dropout)

    # Fully connected layer
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]]) # Reshape conv3 output to fit dense layer input
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1') # Relu activation

    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation

    # Output, class prediction
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64],seed=0)),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128],seed=0)),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256],seed=0)),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024],seed=0)),
    'wd2': tf.Variable(tf.random_normal([1024, 1024],seed=0)),
    'out': tf.Variable(tf.random_normal([1024, 10],seed=0))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([64],seed=0)),
    'bc2': tf.Variable(tf.random_normal([128],seed=0)),
    'bc3': tf.Variable(tf.random_normal([256],seed=0)),
    'bd1': tf.Variable(tf.random_normal([1024],seed=0)),
    'bd2': tf.Variable(tf.random_normal([1024],seed=0)),
    'out': tf.Variable(tf.random_normal([n_classes],seed=0))
}

# Construct model
pred = alex_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    start_time = time.time()
    i = 1;
    while step * batch_size < base_training_iters*i:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            #print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
        if(step * batch_size > base_training_iters*i - 2*batch_size) and (step * batch_size < base_training_iters*i):
            duration = time.time() - start_time
            print(itr_num(base_training_iters*i),", {}".format(duration),",",sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))
            #print('Number of iterations=',itr_num(base_training_iters*i))
            #print("Time taken for training={}".format(duration))
            # Calculate accuracy for 256 mnist test images
            #print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))
            #print('Number of iterations=',itr_num(base_training_iters*i))
            #print("Time taken for training={}".format(duration))
            # Calculate accuracy for 256 mnist test images
            #print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))
            i *= 1.6
            if(i>500):
                #Now, save the graph
                duration = time.time() - start_time
                print(itr_num(base_training_iters*i),", {}".format(duration),",",sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))
                saver.save(sess,'E:\my_test_model')
                break;

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
200 , 0.23363971710205078 , 0.0507813
300 , 0.42515039443969727 , 0.132813
500 , 0.5544886589050293 , 0.328125
800 , 0.7159233093261719 , 0.324219
1000 , 0.9926271438598633 , 0.488281
2000 , 1.4959962368011475 , 0.433594
3000 , 2.154747247695923 , 0.683594
5000 , 3.160388469696045 , 0.816406
9000 , 4.770694017410278 , 0.835938
10000 , 7.288390874862671 , 0.886719
20000 , 11.336151599884033 , 0.902344
40000 , 17.756208658218384 , 0.925781
60000 , 28.114715814590454 , 0.953125
90000 , 44.55744504928589 , 0.964844
100000 , 44.591541051864624 , 0.964844


In [8]:
saver = tf.train.Saver()

#Now, save the graph
saver.save(sess, 'my_test_model',global_step=100000)

#Restore the model:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('my_test_model-100000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))

ValueError: Parent directory of my_test_model doesn't exist, can't save.

In [36]:
step*batch_size, base_training_iters*i - 2*batch_size

(89984, 144115.188075856)